# Segmenting and Clustering - NYC Example

This is an example that illustrates how to convert addresses into latitude and longitude.

It then uses the foursquare API to do some analysis on New York City neighborhoods.

Using some of the foursquare data we will take an analytical approach with k-Means clustering we will segment the city into neighborhoods.

We will also make use of folium to visualize some of the results.

In [1]:
# Importing libraries, etc, that we will use

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [2]:
# Load the new york json data

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
# Define a variable for the features in the newyork_data.json file
neighborhoods_data = newyork_data['features']

# Take a look at the first entry here
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [4]:
# Transform the data into a pandas dataframe
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']
neighborhoods = pd.DataFrame(columns = column_names)

# Loop through the data and fill the data frame
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    neighborhoods = neighborhoods.append({'Borough': borough, 'Neighborhood': neighborhood_name, 'Latitude': neighborhood_lat, 'Longitude': neighborhood_lon}, ignore_index = True)

# Check that the data loaded the way we expected
neighborhoods.head()    

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## Using geopy library

Next we use the geopy library to get the latitude and longitude of NYC. We create an instance of the geocoder and define a user_agent, which we will name ny_explorer.

In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The geographical coordinates of New York City are {}, {}.".format(latitude, longitude))

The geographical coordinates of New York City are 40.7127281, -74.0060152.


In [6]:
# Use folium to create a map of the city

map_newyork = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'],neighborhoods['Neighborhood']):
    label = '{},{}'.format(neighborhood, borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_newyork)

map_newyork

In [7]:
# Simplify the map by restricting view to manhattan

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop = True)

address = 'Manhattan, NY'

geolocator = Nominatim(user_agent = "ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Manhattan are {}, {}'.format(latitude, longitude))

The geographical coordinates of Manhattan are 40.7896239, -73.9598939


In [8]:
map_manhattan = folium.Map(location=[latitude,longitude], zoom_start = 11)

for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#31866.cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_manhattan)
    
map_manhattan

## Using the Foursquare API to explore and segment neighborhoods

In [9]:
# Connecting to foursquare

CLIENT_ID = 'P4GV2G4LEKK4XMTHM0H4H5W3CFR055TAHR2IQ3JPLJ0TFHEK'
CLIENT_SECRET = 'CJ2YTC5JCO4F3PNHKPSZPMFOTIU5RYD4QD1WB5FYRCS4FD5B'
VERSION ='20180604'
LIMIT = 100

In [10]:
manhattan_data.loc[0,'Neighborhood']

'Marble Hill'

In [11]:
neighborhood_latitude = manhattan_data.loc[0,'Latitude']
neighborhood_longitude = manhattan_data.loc[0,'Longitude']
neighborhood_name = manhattan_data.loc[0,'Neighborhood']
print("Latitude and longitude values of {} are {},{}.".format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude values of Marble Hill are 40.87655077879964,-73.91065965862981.


In [12]:
# Get the top 100 venues that are in Marble Hill within a radius of 500 meters

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,CLIENT_SECRET,VERSION,neighborhood_latitude,neighborhood_longitude,radius,LIMIT)

url

'https://api.foursquare.com/v2/venues/explore?&client_id=P4GV2G4LEKK4XMTHM0H4H5W3CFR055TAHR2IQ3JPLJ0TFHEK&client_secret=CJ2YTC5JCO4F3PNHKPSZPMFOTIU5RYD4QD1WB5FYRCS4FD5B&v=20180604&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '602975fc2dbbf54b006e4532'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Marble Hill',
  'headerFullLocation': 'Marble Hill, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 24,
  'suggestedBounds': {'ne': {'lat': 40.88105078329964,
    'lng': -73.90471933917806},
   'sw': {'lat': 40.87205077429964, 'lng': -73.91659997808156}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4429abf964a52037f225e3',
       'name': "Arturo's",
       'location': {'address': '5198 Broadway',
        'crossStreet': 'at 225th St.',
        'lat': 40.87441177110231,
        'lng': -73.91027100981574,
        'labeledLatLngs': [{'label'

In [14]:
# Function to extract the category of a venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
# Clean the json file and put it into a pandas data frame

venues = results['response']['groups'][0]['items']

nearby_venues = pd.json_normalize(venues)

filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [16]:
print("{} venues were returned by Foursquare".format(nearby_venues.shape[0]))

24 venues were returned by Foursquare


## Expanding the Process to All Neighborhoods in Manhattan

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
        
    return(nearby_venues)

In [18]:
names = manhattan_data['Neighborhood']
latitudes = manhattan_data['Latitude']
longitudes = manhattan_data['Longitude']

manhattan_venues = getNearbyVenues(names, latitudes, longitudes)

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [19]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3187, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [20]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,72,72,72,72,72,72
Carnegie Hill,88,88,88,88,88,88
Central Harlem,45,45,45,45,45,45
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,41,41,41,41,41,41
East Village,100,100,100,100,100,100


In [21]:
print("There are {} unique categories.".format(len(manhattan_venues['Venue Category'].unique())))

There are 333 unique categories.


## Neighborhood Analysis

In [22]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood']

fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Stor

In [23]:
manhattan_onehot.shape

(3187, 334)

In [24]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Post Office,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Stor

In [25]:
manhattan_grouped.shape

(40, 334)

In [26]:
# Print top locations in each neighborhood

num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print("\n")

----Battery Park City----
           venue  freq
0          Hotel  0.07
1    Coffee Shop  0.07
2           Park  0.06
3            Gym  0.06
4  Memorial Site  0.04


----Carnegie Hill----
               venue  freq
0        Coffee Shop  0.09
1               Café  0.05
2        Yoga Studio  0.03
3          Bookstore  0.03
4  French Restaurant  0.03


----Central Harlem----
                  venue  freq
0    African Restaurant  0.07
1    Chinese Restaurant  0.04
2   American Restaurant  0.04
3  Gym / Fitness Center  0.04
4                   Bar  0.04


----Chelsea----
                 venue  freq
0          Coffee Shop  0.06
1               Bakery  0.05
2          Art Gallery  0.05
3    French Restaurant  0.04
4  American Restaurant  0.04


----Chinatown----
                venue  freq
0              Bakery  0.08
1  Chinese Restaurant  0.07
2        Dessert Shop  0.04
3   Hotpot Restaurant  0.04
4        Cocktail Bar  0.04


----Civic Center----
                  venue  freq
0           

## Get all of this into a pandas dataframe


In [27]:
# Function to sort the venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
# Display top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st','nd','rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venues'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(manhattan_grouped.iloc[ind,:],num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Hotel,Park,Gym,Memorial Site,Clothing Store,Gourmet Shop,Beer Garden,Food Court,Shopping Mall
1,Carnegie Hill,Coffee Shop,Café,Yoga Studio,Gym,Cosmetics Shop,Pizza Place,Bookstore,French Restaurant,Wine Shop,Bakery
2,Central Harlem,African Restaurant,Chinese Restaurant,Cosmetics Shop,Seafood Restaurant,Gym / Fitness Center,Bar,Art Gallery,American Restaurant,French Restaurant,Library
3,Chelsea,Coffee Shop,Art Gallery,Bakery,American Restaurant,French Restaurant,Ice Cream Shop,Wine Shop,Seafood Restaurant,Market,Bookstore
4,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,Dessert Shop,Hotpot Restaurant,Spa,American Restaurant,Bubble Tea Shop,Salon / Barbershop,Ice Cream Shop


## Neighborhood Clustering

In [29]:
kclusters = 5
manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(manhattan_grouped_clustering)

kmeans.labels_[0:10]

array([3, 0, 0, 0, 0, 1, 1, 3, 0, 1], dtype=int32)

In [30]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Gym,Discount Store,Sandwich Place,Coffee Shop,Yoga Studio,Pizza Place,Steakhouse,Shopping Mall,Seafood Restaurant,Deli / Bodega
1,Manhattan,Chinatown,40.715618,-73.994279,0,Bakery,Chinese Restaurant,Cocktail Bar,Dessert Shop,Hotpot Restaurant,Spa,American Restaurant,Bubble Tea Shop,Salon / Barbershop,Ice Cream Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,0,Café,Bakery,Mobile Phone Shop,Bank,Grocery Store,Deli / Bodega,Gym,Latin American Restaurant,Tapas Restaurant,Italian Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,0,Mexican Restaurant,Lounge,Restaurant,Café,Caribbean Restaurant,Bakery,Chinese Restaurant,Park,Pizza Place,Wine Bar
4,Manhattan,Hamilton Heights,40.823604,-73.949688,0,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Bakery,Park,Cocktail Bar,Sandwich Place,Chinese Restaurant


In [31]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start = 11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat,lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster-1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)

map_clusters
    

In [32]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Bakery,Chinese Restaurant,Cocktail Bar,Dessert Shop,Hotpot Restaurant,Spa,American Restaurant,Bubble Tea Shop,Salon / Barbershop,Ice Cream Shop
2,Washington Heights,Café,Bakery,Mobile Phone Shop,Bank,Grocery Store,Deli / Bodega,Gym,Latin American Restaurant,Tapas Restaurant,Italian Restaurant
3,Inwood,Mexican Restaurant,Lounge,Restaurant,Café,Caribbean Restaurant,Bakery,Chinese Restaurant,Park,Pizza Place,Wine Bar
4,Hamilton Heights,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Bakery,Park,Cocktail Bar,Sandwich Place,Chinese Restaurant
6,Central Harlem,African Restaurant,Chinese Restaurant,Cosmetics Shop,Seafood Restaurant,Gym / Fitness Center,Bar,Art Gallery,American Restaurant,French Restaurant,Library
12,Upper West Side,Italian Restaurant,Café,Bar,Bakery,Indian Restaurant,Wine Bar,Sushi Restaurant,Vegetarian / Vegan Restaurant,Breakfast Spot,Ice Cream Shop
17,Chelsea,Coffee Shop,Art Gallery,Bakery,American Restaurant,French Restaurant,Ice Cream Shop,Wine Shop,Seafood Restaurant,Market,Bookstore
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,Gym,Indian Restaurant,Boutique,American Restaurant,Seafood Restaurant,Cosmetics Shop
19,East Village,Bar,Pizza Place,Mexican Restaurant,Italian Restaurant,Wine Bar,Korean Restaurant,Ice Cream Shop,Salon / Barbershop,Coffee Shop,Ramen Restaurant
20,Lower East Side,Chinese Restaurant,Café,Japanese Restaurant,Grocery Store,Latin American Restaurant,Bakery,Art Gallery,Coffee Shop,Dance Studio,Pharmacy


In [33]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Coffee Shop,Italian Restaurant,Bakery,Exhibit,Gym / Fitness Center,American Restaurant,Yoga Studio,Spa,French Restaurant,Juice Bar
10,Lenox Hill,Italian Restaurant,Cocktail Bar,Coffee Shop,Pizza Place,Sushi Restaurant,Gym,Burger Joint,Gym / Fitness Center,Café,Salon / Barbershop
13,Lincoln Square,Plaza,Theater,Concert Hall,Performing Arts Venue,Café,Gym / Fitness Center,Indie Movie Theater,Wine Shop,Cycle Studio,Cosmetics Shop
14,Clinton,Italian Restaurant,Gym / Fitness Center,Theater,Wine Shop,Coffee Shop,Sandwich Place,American Restaurant,Gym,Spa,Hotel
15,Midtown,Hotel,Coffee Shop,Steakhouse,Theater,Sporting Goods Shop,Clothing Store,Bakery,Gym,Bookstore,American Restaurant
21,Tribeca,Park,Italian Restaurant,American Restaurant,Wine Bar,Spa,Café,Coffee Shop,Steakhouse,Poke Place,Bakery
24,West Village,Italian Restaurant,New American Restaurant,Cocktail Bar,American Restaurant,Wine Bar,Park,Coffee Shop,Sandwich Place,Chinese Restaurant,French Restaurant
29,Financial District,Coffee Shop,Cocktail Bar,Bar,Italian Restaurant,Pizza Place,Park,Gym,Gym / Fitness Center,Mexican Restaurant,Salad Place
32,Civic Center,Coffee Shop,Gym / Fitness Center,Spa,Hotel,Cocktail Bar,Yoga Studio,Italian Restaurant,Bakery,French Restaurant,Park
33,Midtown South,Korean Restaurant,Hotel,Cosmetics Shop,Japanese Restaurant,Café,Burger Joint,Dessert Shop,Hotel Bar,Gym / Fitness Center,Coffee Shop


In [34]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Park,Bar,Coffee Shop,Boat or Ferry,Farmers Market,Gym / Fitness Center,Fountain,Harbor / Marina,Gas Station,Cocktail Bar


In [35]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Seafood Restaurant,Deli / Bodega,Sushi Restaurant,Italian Restaurant,Chinese Restaurant,Mexican Restaurant,Bus Stop,Lounge,Boutique
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Thai Restaurant,Sandwich Place,Spa,Latin American Restaurant,Restaurant,Steakhouse,Beer Bar
9,Yorkville,Italian Restaurant,Coffee Shop,Gym,Deli / Bodega,Sushi Restaurant,Bar,Wine Shop,Japanese Restaurant,Diner,Gym / Fitness Center
11,Roosevelt Island,Park,Playground,Outdoors & Recreation,School,Liquor Store,Supermarket,Dry Cleaner,Soccer Field,Coffee Shop,Greek Restaurant
16,Murray Hill,Coffee Shop,Sandwich Place,Japanese Restaurant,Bar,Hotel,American Restaurant,Burger Joint,Gym / Fitness Center,Deli / Bodega,Grocery Store
26,Morningside Heights,Park,Coffee Shop,Bookstore,American Restaurant,Burger Joint,Café,Farmers Market,Tennis Court,Supermarket,Mediterranean Restaurant
28,Battery Park City,Coffee Shop,Hotel,Park,Gym,Memorial Site,Clothing Store,Gourmet Shop,Beer Garden,Food Court,Shopping Mall
35,Turtle Bay,Coffee Shop,Deli / Bodega,Italian Restaurant,Sushi Restaurant,Seafood Restaurant,Park,Japanese Restaurant,Hotel,Garden,Thai Restaurant
36,Tudor City,Park,Deli / Bodega,Café,Mexican Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,Restaurant,Thai Restaurant,Seafood Restaurant


In [36]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venues,2nd Most Common Venues,3rd Most Common Venues,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Gym,Discount Store,Sandwich Place,Coffee Shop,Yoga Studio,Pizza Place,Steakhouse,Shopping Mall,Seafood Restaurant,Deli / Bodega
